This script automates the generation and processing of a dataset for creating SMART goals from vague goals across multiple professional improvement areas. Using the GPT-4o-mini model via OpenAI's API, it ensures high-quality and contextually relevant goal generation. The workflow includes generating goals, extracting a sample for manual evaluation, and producing additional goals for dataset enhancement, making it a robust pipeline for training and evaluation.

In [ ]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.0 MB/s eta 0:00:00


Section 1: Goal Generation:This section generates vague goals and their corresponding SMART goals across 17 areas of professional improvement using the GPT-4o-mini model. Each area produces 80 goal pairs, ensuring diverse and contextually relevant outputs that align with SMART criteria. Retry logic is implemented to handle API rate limits and maintain data quality.

In [ ]:
import openai
import pandas as pd
import time
import csv

# Set your OpenAI API key
openai.api_key = 'sk-Ew0WE_19KVUrj3IwpBa9idz_aPo33-oxgXuy_nsVXRT3BlbkFJWa8gkEXExwWPg0Yr8Cde5wbcaIjPVDAppHspv8hKIA'

# Define areas of improvement with one-word focus
areas_of_improvement = [
    "leadership",
    "strategic vision",
    "communication",
    "influencing",
    "emotional intelligence",
    "relationships",
    "innovation",
    "adaptability",
    "collaboration",
    "ethical decision making",
    "teamwork",
    "analytic decision making",
    "negotiation skills",
    "entrepreneurial",
    "technical skills",
    "personal brand",
    "time management"
]

# Function to generate vague and SMART goals
def generate_goals(area):
    max_retries = 5  # Maximum number of retries for API calls

    for attempt in range(max_retries):
        try:
            # Start a conversation to generate a simple, vague goal with an ethical focus
            vague_response = openai.ChatCompletion.create(
                model="gpt-4o-mini",  # Specify the chat model
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": f"Generate a simple, vague personal intent related to improving {area} in a professional context. Avoiding outcome-based language. Keep it open-ended and colloquial."}
                ]
            )
            vague_goal = vague_response['choices'][0]['message']['content'].strip()

            # Delay to avoid rate limit issues
            time.sleep(2)

            # Continue the conversation to generate a SMART goal with the specific format
            smart_response = openai.ChatCompletion.create(
                model="gpt-4o-mini",  # Use the same model for consistency
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": f"Based on the following vague goal, create a SMART goal that is creative, in a straightforward narrative style of writing. Ensure the SMART goal is: \n- Specific: Clearly defined and focused. \n- Measurable: Outcomes can be quantitatively or qualitatively assessed. \n- Achievable: Consider available resources, potential challenges, do not presume context and adjust the scope to ensure success. \n- Relevant: Directly aligns with broader personal growth, explaining why this goal matters. \n- Time-bound: Includes a specific deadline, avoid overly rigid timelines that ignore unforeseen changes.\nPlease do not restate the definitions of SMART criteria in the goal itself.\nVague Goal: '{vague_goal}'."}
                ]
            )
            smart_goal = smart_response['choices'][0]['message']['content'].strip()
            return vague_goal, smart_goal

        except openai.error.RateLimitError as e:
            # Access the Retry-After header if available
            retry_after = int(e.headers.get("Retry-After", 10))  # Use the value from the header, or default to 10 seconds
            print(f"Rate limit exceeded, retrying in {retry_after} seconds... Attempt {attempt + 1} of {max_retries}")
            time.sleep(retry_after)  # Wait for the specified retry time
        except openai.error.OpenAIError as e:
            print(f"An OpenAI error occurred: {e}")
            return None, None
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            return None, None

    print("Failed to generate goals after several retries.")
    return None, None

# Generate goals and save to DataFrame
goals_data = []
for area in areas_of_improvement:
    for i in range(80):
        vague_goal, smart_goal = generate_goals(area)
        if vague_goal and smart_goal:
            goals_data.append({
                'Area': area,
                'Vague Goal': vague_goal,
                'SMART Goal': smart_goal
            })
            print(f"Generated {i+1}/80 goals for {area}")
        else:
            print(f"Skipping generation for {area} at index {i} due to errors.")

# Save the data to a CSV file
df = pd.DataFrame(goals_data)
df.to_csv('/content/drive/My Drive/SMART_Goals_v2.csv', index=False, sep=',', quoting=csv.QUOTE_ALL, lineterminator='\n')


# Display the first few rows of the DataFrame
print(df.head())
print("80 goals for each area generated and saved successfully.")


Generated 1/80 goals for leadership
Generated 2/80 goals for leadership
Generated 3/80 goals for leadership
Generated 4/80 goals for leadership
Generated 5/80 goals for leadership
Generated 6/80 goals for leadership
Generated 7/80 goals for leadership
Generated 8/80 goals for leadership
Generated 9/80 goals for leadership
Generated 10/80 goals for leadership
Generated 11/80 goals for leadership
Generated 12/80 goals for leadership
Generated 13/80 goals for leadership
Generated 14/80 goals for leadership
Generated 15/80 goals for leadership
Generated 16/80 goals for leadership
Generated 17/80 goals for leadership
Generated 18/80 goals for leadership
Generated 19/80 goals for leadership
Generated 20/80 goals for leadership
Generated 21/80 goals for leadership
Generated 22/80 goals for leadership
Generated 23/80 goals for leadership
Generated 24/80 goals for leadership
Generated 25/80 goals for leadership
Generated 26/80 goals for leadership
Generated 27/80 goals for leadership
Generated 

## goal generation with updated prompts

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.0 MB/s eta 0:00:00


Section 2: Dataset Sampling
This section extracts a random 20% sample from the generated dataset for external evaluation by a coach. The sampling is reproducible through a fixed random seed, ensuring consistent subsets for manual review. The sampled data is saved to a separate file for easy access and distribution.

In [ ]:

# Load the generated dataset if not already loaded
df = pd.read_csv('/content/drive/My Drive/SMART_Goals_v2.csv')

# Extract 20% of the data randomly
df_sample = df.sample(frac=0.2, random_state=42)  # random_state ensures reproducibility

# Save the 20% sample to a new CSV file for your coach
df_sample.to_csv('/content/drive/My Drive/SMART_Goals_Sample_for_Coach.csv', index=False, sep=',', quoting=csv.QUOTE_ALL, lineterminator='\n')

#  Display the first few rows of the sample
print(df_sample.head())


                 Area                                         Vague Goal  \
51         leadership  I'm thinking about ways to enhance my approach...   
1101  entrepreneurial  I’m looking to explore ways to enhance my appr...   
1313  time management  I’m looking to explore some ways to get a bett...   
514        innovation  I'm thinking about ways to stir up some fresh ...   
1075  entrepreneurial  I’m looking to explore ways to enhance my entr...   

                                             SMART Goal  
51    Over the next three months, I will enhance my ...  
1101  In the next six months, I will enhance my appr...  
1313  To gain better control over my work schedule a...  
514   In order to cultivate an environment rich in f...  
1075  By the end of the next three months, I will en...  


Section 3: Additional Goal Generation
This section generates supplementary goals to enhance the dataset further. The new goals are structured similarly to the primary dataset, ensuring consistency and expanding the variety of available examples for training and evaluation. There are different araes for improvment added, in order to increase diversity.

In [ ]:

import openai
import pandas as pd
import time
import csv

# Set your OpenAI API key
openai.api_key = 'sk-Ew0WE_19KVUrj3IwpBa9idz_aPo33-oxgXuy_nsVXRT3BlbkFJWa8gkEXExwWPg0Yr8Cde5wbcaIjPVDAppHspv8hKIA'

# Define areas of improvement ( 5 new areas)
areas_of_improvement = [

    # New areas
    "mentorship",
    "creativity",
    "diversity",
    "resilience",
    "project management"
]

# Function to generate vague and SMART goals (same as before)
def generate_goals(area):
    max_retries = 5  # Maximum number of retries for API calls

    for attempt in range(max_retries):
        try:
            # Start a conversation to generate a simple, vague goal with an ethical focus
            vague_response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": f"Generate a simple, vague personal intent related to improving {area} in a professional context. Avoiding outcome-based language. Keep it open-ended and colloquial."}
                ]
            )
            vague_goal = vague_response['choices'][0]['message']['content'].strip()

            # Delay to avoid rate limit issues
            time.sleep(2)

            # Continue the conversation to generate a SMART goal with the specific format
            smart_response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": f"Based on the following vague goal, create a SMART goal that is creative, in a straightforward narrative style of writing. Ensure the SMART goal is: \n- Specific: Clearly defined and focused. \n- Measurable: Outcomes can be quantitatively or qualitatively assessed. \n- Achievable: Consider available resources, potential challenges, do not presume context and adjust the scope to ensure success. \n- Relevant: Directly aligns with broader personal growth, explaining why this goal matters. \n- Time-bound: Includes a specific deadline, avoid overly rigid timelines that ignore unforeseen changes.\nPlease do not restate the definitions of SMART criteria in the goal itself.\nVague Goal: '{vague_goal}'."}
                ]
            )
            smart_goal = smart_response['choices'][0]['message']['content'].strip()
            return vague_goal, smart_goal  # Return results if successful

        except openai.error.RateLimitError as e:
            retry_after = int(e.headers.get("Retry-After", 10))  # Use the value from the header, or default to 10 seconds
            print(f"Rate limit exceeded, retrying in {retry_after} seconds... Attempt {attempt + 1} of {max_retries}")
            time.sleep(retry_after)  # Wait for the specified retry time
        except openai.error.OpenAIError as e:
            print(f"An OpenAI error occurred: {e}")
            return None, None  # Return None if a different OpenAI error occurs
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            return None, None  # Return None if an unexpected error occurs

    print("Failed to generate goals after several retries.")
    return None, None  # Return None if all retries fail

# Generate goals and save to DataFrame
goals_data = []
for area in areas_of_improvement:
    for i in range(80):
        vague_goal, smart_goal = generate_goals(area)
        if vague_goal and smart_goal:  # Ensure goals were generated
            goals_data.append({
                'Area': area,
                'Vague Goal': vague_goal,
                'SMART Goal': smart_goal
            })
            print(f"Generated {i+1}/80 goals for {area}")
        else:
            print(f"Skipping generation for {area} at index {i} due to errors.")

# Save the data to a CSV file
df = pd.DataFrame(goals_data)
df.to_csv('/content/drive/My Drive/SMART_Goals_extra_v3.csv', index=False, sep=',', quoting=csv.QUOTE_ALL, lineterminator='\n')

# Display the first few rows of the DataFrame
print(df.head())
print("80 goals for each area generated and saved successfully.")




Generated 1/80 goals for mentorship
Generated 2/80 goals for mentorship
Generated 3/80 goals for mentorship
Generated 4/80 goals for mentorship
Generated 5/80 goals for mentorship
Generated 6/80 goals for mentorship
Generated 7/80 goals for mentorship
Generated 8/80 goals for mentorship
Generated 9/80 goals for mentorship
Generated 10/80 goals for mentorship
Generated 11/80 goals for mentorship
Generated 12/80 goals for mentorship
Generated 13/80 goals for mentorship
Generated 14/80 goals for mentorship
Generated 15/80 goals for mentorship
Generated 16/80 goals for mentorship
Generated 17/80 goals for mentorship
Generated 18/80 goals for mentorship
Generated 19/80 goals for mentorship
Generated 20/80 goals for mentorship
Generated 21/80 goals for mentorship
Generated 22/80 goals for mentorship
Generated 23/80 goals for mentorship
Generated 24/80 goals for mentorship
Generated 25/80 goals for mentorship
Generated 26/80 goals for mentorship
Generated 27/80 goals for mentorship
Generated 